In [ ]:
# 0 -> 36 
%cd ..
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Wall Time per Timestep and Episode

In [ ]:
import time
import numpy as np
np.random.seed(42)

from sustaingym.envs.evcharging import GMMsTraceGenerator, EVChargingEnv, RealTraceGenerator
NUM_RUNS = 100

run_time_per_1000 = {}
avg_eps_length = {}

for site in ['caltech']: #, 'jpl']:
    for action_type in ['discrete', 'continuous']:
        for project_action in [False, True]:
            for real in [True, False]:
                start = time.time()

                if real:
                    gen = RealTraceGenerator(site=site, date_period=('2019-05-01', '2019-08-31'))
                else:
                    gen = GMMsTraceGenerator(site=site, date_period=('2019-05-01', '2019-08-31'), n_components=50)

                env = EVChargingEnv(gen, action_type=action_type, project_action=project_action)

                setup_time = time.time() - start
                print(f"EVChargingEnv: site == {site}, action_type == {action_type}, project_action == {project_action}, using real traces == {real}")
                print(f"Time for imports and initializing gyms: {setup_time:.2f} s")

                all_rewards = []
                episode_length = []

                run_time_start = time.time()
                for _ in range(NUM_RUNS):
                    observation = env.reset()
                    done, rewards, episode_timesteps = False, 0., 0.

                    while not done:
                        action = env.action_space.sample()  # random action
                        observation, reward, done, info = env.step(action)

                        rewards += reward
                        episode_timesteps += 1

                    all_rewards.append(rewards)
                    episode_length.append(episode_timesteps)
                run_time = time.time() - run_time_start

                all_rewards = np.array(all_rewards)
                episode_length = np.array(episode_length)

                run_time_per_1000[(site, action_type, project_action, real)] = run_time * 1000 / np.sum(episode_length)
                avg_eps_length[(site, action_type, project_action, real)] = np.mean(episode_length)

                print(f"Run time: {run_time:.2f} s")
                print(f"Run time / episode: {run_time / NUM_RUNS:.2f} s")
                print(f"Run time / 1000 timesteps: {run_time * 1000 / np.sum(episode_length):.2f} s")
                print(f"Reward for random agent: {np.mean(all_rewards):.2f} +/- {np.std(all_rewards):.2f}")
                print(f"Episode length: {np.mean(episode_length):.2f} +/- {np.std(episode_length):.2f} timesteps\n")

                env.close()


In [ ]:
import pandas as pd

idx = pd.MultiIndex.from_tuples(run_time_per_1000.keys(), names=['site', 'action_type', 'project_action', 'real_traces'])

run_times_per_1000 = pd.Series(run_time_per_1000.values(), index=idx)
avg_eps_length = pd.Series(avg_eps_length.values(), index=idx)

pd.DataFrame([run_times_per_1000, avg_eps_length])

display(run_times_per_1000)
display(avg_eps_length)

In [ ]:
df = pd.DataFrame([run_time_per_1000, avg_eps_length], index=['run_times_per_1000', 'avg_eps_length'])
df
df.to_csv('./logs/wall_time/times.csv', index=True)

In [ ]:
pd.read_csv('./logs/wall_time/times.csv', index_col=0)

In [ ]:
df[('caltech', 'continuous', True, True)].loc['run_times_per_1000']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})


In [ ]:
labels = [
    'RT w/ AP', 'RT w/o AP',
    'GS w/ AP', 'GS w/o AP',
]

run_times_cont = [
    df[('caltech', 'continuous', True, True)].loc['run_times_per_1000'],
    df[('caltech', 'continuous', False, True)].loc['run_times_per_1000'],
    df[('caltech', 'continuous', True, False)].loc['run_times_per_1000'],
    df[('caltech', 'continuous', False, False)].loc['run_times_per_1000'],

]
run_times_disc = [
    df[('caltech', 'discrete', True, True)].loc['run_times_per_1000'],
    df[('caltech', 'discrete', False, True)].loc['run_times_per_1000'],
    df[('caltech', 'discrete', True, False)].loc['run_times_per_1000'],
    df[('caltech', 'discrete', False, False)].loc['run_times_per_1000'],
]

x = np.arange(4)  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, run_times_cont, width, label='continuous actions')
rects2 = ax.bar(x + width/2, run_times_disc, width, label='discrete actions')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Time (s)')
ax.set_title('Wall Time / 1,000 Timesteps: Caltech 05/01/2019 - 08/31/2019')
ax.set_xticks(x, labels)
ax.legend(bbox_to_anchor=[1, 0.5])

ax.annotate('RT: real trace', xy=[-0.25, 14])
ax.annotate('GS: GMM samples', xy=[-0.25, 13])
ax.annotate('AP: action projection', xy=[-0.25, 12])

# ax.bar_label(rects1, padding=3)
# ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()